In [ ]:

!pip install pdf2image
!apt-get install poppler-utils
!pip install Pillow



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


KeyboardInterrupt: 

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving batch_1.zip to batch_1.zip


In [17]:
import zipfile
import os
from pathlib import Path
from pdf2image import convert_from_path
from PIL import Image

# Définir les chemins
zip_path = 'batch_1.zip'  # Le fichier ZIP téléchargé
output_folder = '/content/output_images'  # Dossier cible

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)

# Décompresser le fichier ZIP
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_folder)
    print(f"✅ Extraction de {zip_path} réussie dans {output_folder}")
except zipfile.BadZipFile:
    print(f"❌ Erreur : Le fichier {zip_path} est un fichier zip invalide.")
    exit(1)
except Exception as e:
    print(f"❌ Erreur lors de l'extraction : {e}")
    exit(1)

# Parcourir tous les fichiers extraits
for root, dirs, files in os.walk(output_folder):
    for file_name in files:
        file_path = os.path.join(root, file_name)

        # Vérifier si c'est une image
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.tif')):
            print(f"🖼️ Image acceptée : {file_path}")

        # Vérifier si c'est un PDF
        elif file_name.lower().endswith('.pdf'):
            print(f"📄 Conversion du PDF : {file_path}")
            try:
                images = convert_from_path(file_path, dpi=300)
                for i, image in enumerate(images):
                    image_filename = f"{Path(file_name).stem}_page_{i+1}.png"
                    image.save(os.path.join(root, image_filename), 'PNG')
                print(f"✅ Conversion de {file_name} terminée.")
                # os.remove(file_path)  # Décommentez si vous voulez supprimer les PDF
            except Exception as e:
                print(f"❌ Erreur lors de la conversion de {file_path} : {e}")

        else:
            print(f"📁 Fichier non supporté (ignoré) : {file_path}")


✅ Extraction de batch_1.zip réussie dans /content/output_images
📁 Fichier non supporté (ignoré) : /content/output_images/batch_1/invoice_565.pdf0_preprocessed.txt
🖼️ Image acceptée : /content/output_images/batch_1/invoice_466.pdf0_preprocessed.png
📁 Fichier non supporté (ignoré) : /content/output_images/batch_1/invoice_115.pdf0_preprocessed.txt
🖼️ Image acceptée : /content/output_images/batch_1/invoice_504.pdf0.jpg
📁 Fichier non supporté (ignoré) : /content/output_images/batch_1/invoice_13.pdf0_preprocessed.txt
🖼️ Image acceptée : /content/output_images/batch_1/invoice_75_color_B_249.pdf0.jpg
🖼️ Image acceptée : /content/output_images/batch_1/invoice_877.pdf0.jpg
🖼️ Image acceptée : /content/output_images/batch_1/invoice_882.pdf0_preprocessed.png
🖼️ Image acceptée : /content/output_images/batch_1/invoice_272_color_B_246.pdf0_preprocessed.png
📁 Fichier non supporté (ignoré) : /content/output_images/batch_1/invoice_368_color_B_240.pdf0_preprocessed.txt
📁 Fichier non supporté (ignoré) : /

In [ ]:
!pip install opencv-python-headless


In [ ]:
import cv2
import numpy as np
import os

def preprocess_image(image_path):
    try:
        # Lire l’image en niveaux de gris
        gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Binarisation avec un seuillage adaptatif
        binary = cv2.adaptiveThreshold(
            gray, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            15, 10
        )

        # Trouver les coordonnées des pixels non noirs
        coords = np.column_stack(np.where(binary > 0))
        angle = 0

        if len(coords) > 0:
            rect = cv2.minAreaRect(coords)
            angle = rect[-1]

            if angle < -45:
                angle = -(90 + angle)
            else:
                angle = -angle

        # Redressement (deskew)
        (h, w) = gray.shape
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        deskewed = cv2.warpAffine(gray, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        # Re-seuillage après redressement
        final_binary = cv2.adaptiveThreshold(
            deskewed, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            15, 10
        )

        # Nettoyage du bruit (filtre médian)
        cleaned = cv2.medianBlur(final_binary, 3)

        # Sauvegarde de l'image prétraitée
        preprocessed_path = image_path.replace('.png', '_preprocessed.png')
        cv2.imwrite(preprocessed_path, cleaned)

        print(f"✅ Prétraitement terminé : {preprocessed_path}")

    except Exception as e:
        print(f"❌ Erreur lors du prétraitement de {image_path} : {e}")


In [ ]:
input_folder = '/content/output_images'

for filename in os.listdir(input_folder):
    if filename.lower().endswith('.png'):
        image_path = os.path.join(input_folder, filename)
        preprocess_image(image_path)


In [ ]:
# Installer OpenCV si nécessaire
!pip install opencv-python-headless --quiet

# Imports
import os
import cv2
import numpy as np

# Fonction de prétraitement
def preprocess_image(image_path):
    try:
        gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        binary = cv2.adaptiveThreshold(
            gray, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            15, 10
        )

        coords = np.column_stack(np.where(binary > 0))
        angle = 0
        if len(coords) > 0:
            rect = cv2.minAreaRect(coords)
            angle = rect[-1]
            if angle < -45:
                angle = -(90 + angle)
            else:
                angle = -angle

        (h, w) = gray.shape
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        deskewed = cv2.warpAffine(gray, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

        final_binary = cv2.adaptiveThreshold(
            deskewed, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            15, 10
        )

        cleaned = cv2.medianBlur(final_binary, 3)

        preprocessed_path = image_path.replace('.png', '_preprocessed.png') \
                                      .replace('.jpg', '_preprocessed.png') \
                                      .replace('.jpeg', '_preprocessed.png') \
                                      .replace('.tif', '_preprocessed.png') \
                                      .replace('.tiff', '_preprocessed.png')

        cv2.imwrite(preprocessed_path, cleaned)
        print(f"✅ Prétraitement terminé : {preprocessed_path}")

    except Exception as e:
        print(f"❌ Erreur lors du prétraitement de {image_path} : {e}")

# Dossier contenant les images extraites
output_folder = '/content/output_images'

# Appliquer le prétraitement aux images valides
for root, dirs, files in os.walk(output_folder):
    for file_name in files:
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.tif')) \
           and '_preprocessed' not in file_name:
            image_path = os.path.join(root, file_name)
            preprocess_image(image_path)


✅ Prétraitement terminé : /content/output_images/batch_1/invoice_504.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_75_color_B_249.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_877.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_765.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_685.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_477.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_466_color_B_241.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_443.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_307.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1/invoice_98_color_B_248.pdf0_preprocessed.png
✅ Prétraitement terminé : /content/output_images/batch_1

In [16]:
!apt-get install tesseract-ocr -y
!pip install pytesseract --quiet
!pip install Pillow --quiet

import pytesseract
from PIL import Image
import os

# Optionnel : spécifier la langue si vous avez besoin de français
# !apt-get install tesseract-ocr-fra -y

# Fonction d'extraction de texte avec OCR
def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)

        # Modifier la langue ici si le texte est en français : lang='fra'
        text = pytesseract.image_to_string(img, lang='eng')

        # Générer le chemin du fichier texte
        text_path = image_path.rsplit('.', 1)[0] + '.txt'

        with open(text_path, 'w', encoding='utf-8') as f:
            f.write(text)

        print(f"📝 OCR terminé : texte extrait dans {text_path}")

    except Exception as e:
        print(f"❌ Erreur lors de l'OCR de {image_path} : {e}")

# Appliquer l’OCR uniquement aux images prétraitées
for root, dirs, files in os.walk(output_folder):
    for file_name in files:
        if '_preprocessed' in file_name.lower() and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(root, file_name)
            extract_text_from_image(image_path)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
📝 OCR terminé : texte extrait dans /content/output_images/batch_1/invoice_466.pdf0_preprocessed.txt
📝 OCR terminé : texte extrait dans /content/output_images/batch_1/invoice_882.pdf0_preprocessed.txt
📝 OCR terminé : texte extrait dans /content/output_images/batch_1/invoice_272_color_B_246.pdf0_preprocessed.txt
📝 OCR terminé : texte extrait dans /content/output_images/batch_1/invoice_439.pdf0_preprocessed.txt
📝 OCR terminé : texte extrait dans /content/output_images/batch_1/invoice_234_color_B_254.pdf0_preprocessed.txt
📝 OCR terminé : texte extrait dans /content/output_images/batch_1/invoice_29.pdf0_preprocessed.txt
📝 OCR terminé : texte extrait dans /content/output_images/batch_1/invoice_37.pdf0_preprocessed.txt
📝 OCR terminé : texte extrait dans /content/output_ima

In [30]:
output_folder = '/content/output_images'


In [31]:
import os

for root, dirs, files in os.walk('/content/output_images'):
    for file in files:
        if file.endswith('.txt'):
            print(os.path.join(root, file))


/content/output_images/batch_1/invoice_565.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_115.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_13.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_368_color_B_240.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_909.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_821.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_445.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_216.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_596.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_557.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_408_color_B_240.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_703.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_107_color_B_248.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_657.pdf0_preprocessed.txt
/content/output_images/batch_1/invoice_127.pdf0_preproc

In [32]:
text_file_path = '/content/output_images/batch_1/invoice_989.pdf0_preprocessed.txt'

with open(text_file_path, 'r', encoding='utf-8') as file:
    extracted_text = file.read()
print(extracted_text)


Invoice no: 53391878

Date of issue: 09/06/2017
7) Seller: Client:
Turner-Thompson Conner, Price and Melendez
05260 Adkins Key Apt. 283 3540 Paula Gateway
Port Jenniferland, Ml 66016 Jacobview, NC 10439
Tax Id: 985-86-5356 Tax Id: 974-77-1014

IBAN: GB13UVWA89592640537716

7) ITEMS

 

No. Description Qty UM Net price Net worth VAT [%] Gross
worth

Oomphies Boys Ankle Boots
Shoes Gray Green Kids Size 2
Euc

 

Jordan B'Loyal Boys Grade
School Sneakers US 3Y

 

Nike Kid's Air Jordan 4 Retro
‘Laser’ 7y White/Gray
705334-105

New Nike Air Max 90 Aura Blue
Sapphire White Kid's Size 4-7
Sneakers CT1966-400

 

______] SUMMARY

VAT [%] Net worth VAT Gross worth

10% 1011,95 101,20 1113,15

 

$1011,95 $ 101,20 $1113,15

 

 

